In [247]:
from numpy import *
from functools import reduce

In [125]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

def createVocabList(dataSet):
    ls = []
    ls = set([])
    for row in dataSet:
        ls = ls | set(row)
    return list(ls)

def setOfWords2Vec(doc, vocabList):
    vec = [0] * len(vocabList)
    words = doc.split(' ')
    for word in words:
        if word in vocabList:
            vec[vocabList.index(word)] = 1
    return vec


In [126]:
postings, classVec = loadDataSet()
vocabList = createVocabList(postings)

In [87]:
def printVec(vec):
    print('Match words:')
    for i in range(len(vec)):
        v = vec[i]
        if v == 1:
            print(vocabList[i])

In [248]:
printVec(setOfWords2Vec('my dog has flea abc help please', vocabList))

Match words:
my
has
flea
please
help
dog


# 朴素贝叶斯

### 函数定义：`#trainNB(...)`
- trainMatrix: 训练数据矩阵（邮件内容，每行代表一封邮件）
- trainCategory: 训练数据分类标签，在这里标识了矩阵中对应下标的邮件是否垃圾邮件（1 or 0），其值是通过人工辨别标识的


### 算法详述

$p(c)$ 表示标签为 $c$ 的邮件出现的概率，例如 $p(1)$ 表示标签值为 $1$ 的类别概率（在这里我们将垃圾邮件标识为 $1$，非垃圾邮件标识为 $0$，因此是一个`二元分类器`）。

计算 $p(1)$ 的值很简单，标签为 $1$ 的邮件数量除以总数量便是。因为是二元分类，显然 $p_0 = 1 - p_1$（归一性）

假定所有影响结果 $c$ 的条件之间互相独立（所以才叫**朴素**），因此 $p(w|c) = p(w_1|c)p(w_2|c)...p(w_n|c)$；其中 $w$ 是条件向量，$w_i$ 表示第 $i$ 个分向量

我们最终要求的概率实际是 $p(c|w)$，这是一个后验概率，其现实意义是**已知邮件满足条件向量 $w$ 的前提下，它属于分类 $c$ 的概率**。至于条件向量 $w$ 是什么？可以简单理解为邮件具备的一些特点（在这个讨论中是不同的单词），这些特点往往较容易确认（如邮件具备或不具备哪些单词），于是所有这些特点的 $w_i$ 都确定后，所组成的向量 $(w_1, w_2,...,w_n)$ 便是一个确定的条件向量 $w$。于是我们上述的现实意义可以更直白地描述为**已知邮件具备某些单词的前提下，它属于垃圾邮件的概率**

显然这个概率我们无法简单通过统计的数据来直接获得，因此需要借助贝叶斯公式

根据贝叶斯法则，有

$p(c|w) = \frac {p(w|c)p(c)} {p(w)}$

上面我们说了，在假定分向量独立的前提下要求得 $p(w|c)$，只需要分别求得 $p(w_i|c)$ 即可，显然 $p(w_i|c)=n_{w_i|c} / N$；其中 $n_{w_i|c}$ 是在满足条件 $c$ 的情况下条件 $w_i$（也就是邮件中的某个单词）出现的次数，N 则是满足条件 $c$ 的情况下样本空间的总数量。

显然不论是 $n_{w_i|c}$ 还是 $N$ 我们都可以通过遍历所有单词来进行统计，不过这里使用了更巧妙的办法

由于输入的邮件数据是形如以下的矩阵（每一列代表了一个单词，$1$ 表示存在于该邮件中）

```python
[[0,1,0,1],
[1,1,0,1]]
```

如果我们要统计所有单词各自的词频，只需要将每行对应的元素相加即可，上例将得到

```python
[1,2,0,2]
```

如果我们要统计总的词汇量，只需要将所有元素相加即可，上例将得到 $5$


最后我们要获得 $p(w)$，只需要用每个单词的词频除以总词汇量即可，上例将得到

```python
[0.2, 0.4, 0, 0.4]
```

当然，实际使用中还要注意区分每封邮件所属的 $c$

经过上面的计算，我们已经获得了 $p(w|c), p(c)$ ，但我们还不知道 $p(w)$，所以还无法求出 $p(c|w)$ 的具体数值。

不过这并没有关系，因为我们最终要知道的是某封邮件是否垃圾邮件，因此肯定是取概率更大的类别 $c$（也就是说如果 $p(1|w)>p(0|w)$，那么这封邮件将被判定为垃圾邮件）

显然对于任一条件向量 $w$，$p(w)$ 应该是一个常数 $a$。因此有

$p(1|w)p(w)>p(0|w)p(w) \Leftrightarrow ap(1|w)>ap(0|w) \Leftrightarrow p(1|w)>p(0|w)$

可见我们无需获得 $p(w)$ 的具体值也可以判定出是否垃圾邮件，只需要比较 $p(1|c)p(c)$ 与 $p(0|c)p(c)$ 即可


### 其它问题

1. 零概率问题（运用拉普拉斯平滑）
2. 下溢问题（运用对数将乘变为加）



#### 拉普拉斯平滑

解决零概率问题常用的方法。

所谓零概率问题，就是某一概率本不为 0 的事件，但在有限的样本数量下实际观察到的次数为 0，导致其概率在计算的时候被当成了 0 处理。进而导致在一些运算中（如朴素贝叶斯算法）使得所有结果变成了 0。这显然是不合理的，因为没有被实际观察到的事件，也不能认为其出现概率就是 0。

拉普拉斯平滑的做法是：
1. 对每个事件的观测次数加 1
2. 同时都总数加 N，N 为分类数量

例如，假定邮件的词汇表为 cute, stupid, garbage

在给定的邮件中观测到各单词的出现情况如下

```python
[[0,1,0],
 [1,0,0],
 [0,1,1]]
```

对应分类标签为 

```python
[0,1,0]
```

cute, stupid, garbage 在两类标签中出现的次数分别是

```python
1: [1, 0, 0]
0: [0, 2, 1]
total: [1, 2, 1]
p1: [1, 0, 0]
p0: [0, 1, 1]
```

分别加 1 变换后各单词出现情况如下
```python
1: [2, 1, 1]
0: [1, 3, 2]
total: [3,4,3]
p1: [2/3, 1/4, 1/3]
p0: [1/3, 3/4, 2/3]
```

可见变换后的概率仍满足归一性质，但概率变化却比较大。这是因为我们示例用的数据量特别小导致，实际训练算法的时候不可能用这么少数据的，当上面的 total 变得很大时，加 1 对各单词出现概率的影响就变得非常小，使得避免了零概率问题的同时也不会对结果造成太大影响




In [261]:

def trainNB(trainMatrix, trainCategory):
    words = len(trainMatrix[0])
    total = len(trainCategory)
    p1 = len(list(filter(lambda v: v == 1, trainCategory))) / total
    
    # 拉普拉斯平滑，避免 0 概率问题
    p1_counts = ones(words)
    p0_counts = ones(words)
    p1_denom = len(set(trainCategory))  # 加上分类标签数量 N
    p0_denom = len(set(trainCategory))
    for i in range(len(trainMatrix)):
        row = trainMatrix[i]
        if trainCategory[i] == 1:
            # spam mail
            p1_counts += row
            p1_denom += sum(row)
        else:
            p0_counts += row
            p0_denom += sum(row)

    # 返回概率的对数，避免数值下溢
    return log(p1_counts/p1_denom), log(p0_counts/p0_denom), p1

In [232]:
def classifyNB(vec, p1_vec, p0_vec, p1):
    # 如果不存在数值下溢问题，理论上应该采用以下运算
#     is_p1 = reduce(lambda a,b: a*b, array(vec) * p1_vec * p1)
#     is_p0 = reduce(lambda a,b: a*b, array(vec) * p0_vec * (1-p1))
    
    # 由于实际运算中存在下溢问题，因此传入的条件概率向量均为对数处理后的值，因此累乘运算应改为累加
    is_p1 = sum(array(vec) * p1_vec) + log(p1)
    is_p0 = sum(array(vec) * p0_vec) + log(1-p1)
    print(is_p1, '>', is_p0)
    return is_p1 > is_p0

In [252]:
mat = []
for post in postings:
    mat.append(setOfWords2Vec(' '.join(post), vocabList))

In [262]:
p1_vec, p0_vec, p1 = trainNB(mat, classVec)

In [263]:
print(classifyNB(setOfWords2Vec('hello world cute I love', vocabList), p1_vec, p0_vec, p1))
print(classifyNB(setOfWords2Vec('hello world cute I love garbage', vocabList), p1_vec, p0_vec, p1))
print(classifyNB(setOfWords2Vec('hello world cute I love stupid', vocabList), p1_vec, p0_vec, p1))
print(classifyNB(setOfWords2Vec('hello world stupid garbage ', vocabList), p1_vec, p0_vec, p1))

-9.826714493730215 > -8.387995252944556
False
-12.178089750893692 > -11.646091790966038
False
-11.484942570333747 > -11.646091790966038
True
-4.702750514326955 > -7.20934025660291
True
